### Below is the main conditioning program. 

#### Steps:

1. Get VIIRS bounding boxes

2. For each MERRA datapoint:
    1. Get distance from closest VIIRS bounding box - throw out ones that are too far (dist > 40)
    2. Store value of datapoint
    3. getROI of datapoint
    4. gridVIIRS of datapoint - if it's too sparse (i.e. num nonzero > 80%)
    5. Save gridded VIIRS data, metadata of MERRA datapoint
    

In [1]:
import os
import tarfile
import xarray as xr
import numpy as np
import pickle
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [2]:
## Extracts boundary metadata from a particular netcdf file
def extractBounds(filename):
    dataset = xr.open_dataset(filename)
    pp = dataset.geospatial_bounds[9:-2].split(",")
    polygon = []
    center = np.array([0, 0])
    for point in pp:
        pt = np.array(list(map(float, point.split(" "))))
        polygon.append(pt)
        center = center + pt
    np_poly = np.array(polygon)
    
    return {"bounds": np_poly, "btime": dataset.time_coverage_start,
            "etime": dataset.time_coverage_end, "center": center / len(polygon)}

## Squishes the larger tarfiles to make things faster.
def extractNestedTars(folder):
    tar_files = os.listdir(folder)
    for tar_f in tar_files:
        if (tar_f == "metadata" or tar_f == "merra.nc"):
            continue
        print("Extracting " + tar_f)
        tar = tarfile.open(folder + tar_f)
        tar.extractall(path= folder + "extracted/")
        tar.close()

## Gets the boundary metadata of all VIIRS files within a directory
def getVIIRSMetadata(folder):
    metadata = {}
    tar_files = os.listdir(folder)
    
    for tar_f in tar_files:
            
        if (tar_f == "metadata" or tar_f == "merra.nc"):
            continue
        print("Extracting " + tar_f)
        tar = tarfile.open(folder + "/" + tar_f)
        tar.extractall(path="data/curr_data")
        tar.close()
            
        files = os.listdir("data/curr_data")
        for f in files:
            print("Opening " + f)
            
            ## THIS IS WHERE YOU CAN APPLY SOME SORT OF PROCESSING STEP TO ALL THE DATA
            metadata[tar_f + "/" + f] = extractBounds("data/curr_data/" + f)
        
            print("Deleting " + f)
            os.remove("data/curr_data/" + f)
        print("Done extracting " + tar_f)
        
    print("Done extracting")

    with open(folder + "metadata", "wb") as handle:
        pickle.dump(metadata, handle)
        
    return metadata

## Gets the VIIRS file closest to a given point
def getClosestVIIRSSwath(point, metadata):
        m_files = np.array(list(metadata.keys()))
        m_vals = np.array(list(metadata.values()))
        c_points = []
        for vals in m_vals:
            c_points.append(vals["center"])
        # Gets closest point
        c_array = np.asarray(c_points)
        dists = np.sum((c_array - point)**2, axis=1)
        return [m_files[np.argmin(dists)], np.min(dists)]

## Extracts and opens the VIIRS data corresponding to the full filename/path
def getVIIRSData(folder, filename):
    levels = filename.split("/")
    
    print("Extracting " + levels[0])
    tar = tarfile.open(folder + levels[0])
    tar.extractall(path="data/curr_data")
    tar.close()
    
    print("Reading " + levels[1])
    data = xr.open_dataset("data/curr_data/" + levels[1])
    
    for f in os.listdir("data/curr_data/"):
        os.remove("data/curr_data/" + f)
    
    return data

## Converts VIIRS data to an array for linear interpolation.
def getVIIRSArrays(viirs_data):
    sample_points = []
    aod_array = []
    
    aods = viirs_data.AOD550
    aod_array = aods.values.reshape(1, -1)[0]
    lons = aods.Longitude.values.reshape(1, -1)[0]
    lats = aods.Latitude.values.reshape(1, -1)[0]
    sample_points = np.column_stack((lons, lats))
    return [sample_points, aod_array]

## Creates a ROI for the particular MERRA datapoint
def getMERRAROI(lon, lat):
    d_lon = 0.625
    d_lat = 0.5
    polygon = []
    bounds = {"minLon": lon - d_lon, "maxLon": lon + d_lon,
              "minLat": lat - d_lat, "maxLat": lat + d_lat}
    polygon.append(np.array([bounds["minLon"], bounds["minLat"]]))
    polygon.append(np.array([bounds["minLon"], bounds["maxLat"]]))
    polygon.append(np.array([bounds["maxLon"], bounds["maxLat"]]))
    polygon.append(np.array([bounds["maxLon"], bounds["minLat"]]))
    polygon.append(np.array([bounds["minLon"], bounds["minLat"]]))
    return [np.array(polygon), bounds]

## Performs nearest neighbor linear interpolation on the VIIRS data array.
def VIIRStoGrid(s_points, data_array, boundary):
    numLonPoints = 100j
    numLatPoints = 80j
    
    lon_grid, lat_grid = np.mgrid[boundary["minLon"]:boundary["maxLon"]:numLonPoints, 
                                  boundary["minLat"]:boundary["maxLat"]:numLatPoints]
    
    gridded_data = griddata(s_points, data_array, (lon_grid, lat_grid), method="nearest")
    return gridded_data

def getT(m_data, swath_name):
    time = list(map(int, m_data[swath_name]["btime"].split("T")[1][:-1].split(":")))
    if time[1] > 30:
        return time[0] + 1
    else:
        return time[0]

In [3]:
def process(folder):
    metadata = None
    if "metadata" in os.listdir(folder):
        with open(folder + "metadata", "rb") as md_file:
            metadata = pickle.load(md_file)
    else:
        metadata = getVIIRSMetadata(folder)
        
    merra_data = xr.open_dataset(folder + "merra.nc")
    
    prev_swath = ""
    viirs_data = None
    viirs_arrays = None
    print(len(merra_data.lon))
    for y in range(100, len(merra_data.lat)):
        for x in range(100, len(merra_data.lon)):
            # Getting lat/lon of merra gridpoint
            point = [merra_data.lon.values[x], merra_data.lat.values[y]]
            print("\nConsidering merra point at " + str(point) + "\nIndex " + str(x) + ", " + str(y))
            
            # Getting corresponding VIIRs box
            [closest_viirs, distance] = getClosestVIIRSSwath(point, metadata)
            
            # Skipping point if too far from center of swath
            print(distance)
            if (distance > 40):
                print("Too far from center of VIIRS data.")
                continue
            #print("Closest VIIR swath: " + closest_viirs)
            
            # Selecting correct time for observation
            time = getT(metadata, closest_viirs)
            merra_val = merra_data.TOTEXTTAU.values[time][y][x]
            print("MERRA value: " + str(merra_val) + " taken at time " + str(time) + "hrs")
            
            # Updating satellite data if the swath has changed.
            if (prev_swath != closest_viirs):
                print("Retrieving VIIRS data from closest swath.")
                viirs_data = getVIIRSData(folder, closest_viirs)
                viirs_arrays = getVIIRSArrays(viirs_data)
            prev_swath = closest_viirs
            
            # Gets the size of the box around the MERRA point
            print("Getting ROI")
            roi = getMERRAROI(point[0], point[1])
            
            # Interpolates VIIRS data to 100x80 grid. This is the slowest part.
            print("Gridding viirs")
            gridded_viirs = VIIRStoGrid(viirs_arrays[0], viirs_arrays[1], roi[1])
            print("Counting number of points")
            num_points = np.count_nonzero(~np.isnan(gridded_viirs))
            print("Number of points: " + str(num_points))
            if (num_points < 100):
                continue
                
            # Saving the VIIRS data to a pickle file
            with open(folder + "processed/{:04d}_{:04d}".format(x, y), "wb") as datafile:
                pickle.dump({"gridded_viirs":gridded_viirs, "roi": roi,
                             "merra_val":merra_val, "closest_viirs":closest_viirs}, datafile)
                
    

In [ ]:
process("data/VIIRS-20200320/")

576

Considering merra point at [-117.5, -40.0]
Index 100, 100
84.00719943719976
Too far from center of VIIRS data.

Considering merra point at [-116.875, -40.0]
Index 101, 100
74.08839943719978
Too far from center of VIIRS data.

Considering merra point at [-116.25, -40.0]
Index 102, 100
64.9508494371998
Too far from center of VIIRS data.

Considering merra point at [-115.625, -40.0]
Index 103, 100
56.59454943719982
Too far from center of VIIRS data.

Considering merra point at [-115.0, -40.0]
Index 104, 100
49.01949943719984
Too far from center of VIIRS data.

Considering merra point at [-114.375, -40.0]
Index 105, 100
42.22569943719985
Too far from center of VIIRS data.

Considering merra point at [-113.75, -40.0]
Index 106, 100
36.21314943719987
MERRA value: 0.12093724 taken at time 21hrs
Retrieving VIIRS data from closest swath.
Extracting JRR-AOD_v2r1_j01_s202003202121104_e202003202131077_c202003202215140.tar.gz
Reading JRR-AOD_v2r1_j01_s202003202121104_e202003202122350_c20200320